# 1.分词

In [1]:
# 读取文件
import pandas as pd
table = pd.read_excel('poem_v2.xlsx')
word_list_df = pd.read_excel('wordlist_v2.xlsx')

# 剪裁一下表格，说不定能快点？
table['content'] = table['simple']
word_list_df['word'] = word_list_df['simple']
word_list_df = word_list_df[['word']]
word_list_df = word_list_df.drop_duplicates().reset_index()

#yz = -1
#for i, row in word_list_df.iterrows():
#    if row['word'] == '一作':
#        yz = i
#        break
#word_list_df = word_list_df.drop(yz).reset_index()
word_list_df

,index,word
0,0,阿
1,1,阿房
2,2,阿阁
3,3,阿娇
4,4,阿连
...,...,...
21368,21524,座
21369,21525,座上
21370,21526,座隅
21371,21527,座中


In [2]:
# 计算wordlist中词语的最大长度max_length
# TODO
max_length = word_list_df['word'].str.len().max()
max_length

3

In [3]:
# 实现前向最大匹配分词算法
# string: 待分词的串
# word_list: 词表
# max_length: 词表中词语的最大长度
import re
def cut(string, word_list, max_length):
    # TODO
    res = []
    
    # 建立max_length个字典，相同长度的词放一个字典里,用来查找
    word_dict = [{}]
    for i in range(max_length):
        word_dict.append({})
        
    for word in word_list['word']:
        word_dict[len(word)][word] = 1
    
    while len(string) > 0:
        while len(string) > 0 and string[0] in '#$() ':
            string = string[1:]
        if len(string) == 0:
            break
            
        have_match = False
            
        for length in range(max_length, 0, -1):
            if string[:length] in word_dict[length]:
                if(string[:length] != '一作'):
                    res.append(string[:length])
                string = string[length:]
                have_match = True
                break
        
        if not have_match:
            string = string[1:]
       
    return res

cut('##饯唐永昌( 一作饯唐郎中洛阳令)', word_list_df, 3)

['饯', '唐', '永昌', '饯', '唐', '郎中', '洛阳', '令']

In [4]:
# 执行分词（可能需要等待几分钟）
table['words'] = table['content'].apply(lambda x: ' '.join(cut(x, word_list_df, max_length)))
table

,ID,Poem_id,line_number,content,simple,words
0,1,4371,-100,##饯唐永昌( 一作饯唐郎中洛阳令),##饯唐永昌( 一作饯唐郎中洛阳令),饯 唐 永昌 饯 唐 郎中 洛阳 令
1,2,4371,-1,$$沈佺期,$$沈佺期,沈 期
2,3,4371,1,洛阳旧有( 一作出) 神明宰,洛阳旧有( 一作出) 神明宰,洛阳 旧有 出 神明 宰
3,4,4371,2,辇毂由来天地中,辇毂由来天地中,辇毂 由来 天地 中
4,5,4371,3,馀邑政成何足贵,馀邑政成何足贵,馀 邑 政成 何足 贵
...,...,...,...,...,...,...
46272,46273,39205,-1,$$李舜弦,$$李舜弦,李 舜弦
46273,46274,39205,1,饮筵中散酒微醒,饮筵中散酒微醒,饮 筵 中 散 酒 微 醒
46274,46275,39205,2,蒙蒙雨草瑶阶溼,蒙蒙雨草瑶阶溼,蒙蒙 雨 草 瑶阶 溼
46275,46276,39205,3,钟晓愁吟独倚屏,钟晓愁吟独倚屏,钟 晓 愁吟 独倚 屏


# 2.统计每首诗内每个词的TF-IDF值

In [5]:
# 按照空格分开，stack
split_words = table['words'].str.split(' ', expand=True).stack().rename('word').reset_index()
new_data = pd.merge(table[['Poem_id', 'line_number']], split_words, left_index=True, right_on='level_0')

# 不知为啥分词后会有空串，去掉
new_data = new_data[new_data['word'] != '']
new_data

,Poem_id,line_number,level_0,level_1,word
0,4371,-100,0,0,饯
1,4371,-100,0,1,唐
2,4371,-100,0,2,永昌
3,4371,-100,0,3,饯
4,4371,-100,0,4,唐
...,...,...,...,...,...
195801,39205,3,46275,4,屏
195802,39205,4,46276,0,尽日
195803,39205,4,46276,1,池边
195804,39205,4,46276,2,钓


In [6]:
# 计算TF-IDF值
tf_map = new_data['word'].value_counts()
tf = tf_map[word_list_df['word']]
TF_IDF = pd.DataFrame({'TF':tf, 'IDF':0})
TF_IDF = TF_IDF[TF_IDF['TF'] > 0]

'''
full_word_dict = {}
for word in TF_IDF.index:
    full_word_dict[word] = 1 
'''

def count_df(grp):
    for word in grp['word'].unique():
        #if word in full_word_dict:
        TF_IDF.loc[word, 'IDF'] += 1
    
new_data.groupby('Poem_id').apply(count_df)

import numpy as np

poem_num = len(new_data['Poem_id'].unique())

TF_IDF['IDF'] = np.log(poem_num / (TF_IDF['IDF'] + 1)) # 加一进行平滑

TF_IDF['TF*IDF'] = TF_IDF['TF'] * TF_IDF['IDF']

TF_IDF

,TF,IDF,TF*IDF
word,,,
阿,18.0,6.116541,110.097741
阿房,2.0,8.256607,16.513215
阿阁,1.0,8.256607,8.256607
阿娇,5.0,7.157995,35.789975
阿连,1.0,8.256607,8.256607
...,...,...,...
座,6.0,7.003844,42.023066
座上,5.0,7.157995,35.789975
座隅,1.0,8.256607,8.256607


In [7]:
# 计算每首诗的TF-IDF值

word_num = TF_IDF.shape[0]
poem_num = new_data['Poem_id'].nunique()

TF_IDF_pp = np.zeros((poem_num, word_num))

word2id = {}
i = 0
for word in TF_IDF.index:
    word2id[word] = i
    i = i + 1

i = 0
def cnt_tf_pp(poem):
    global i
    for word in poem['word']:
        TF_IDF_pp[i, word2id[word]] += 1
    i += 1

new_data.groupby('Poem_id').apply(cnt_tf_pp)

IDF = list(TF_IDF['IDF'])

TF_IDF_pp = TF_IDF_pp.T
for (i, line) in enumerate(TF_IDF_pp):
    line *= IDF[i]
TF_IDF_pp = TF_IDF_pp.T

In [8]:
# 保存
import pickle
with open('./var/word2id', 'wb') as f:
    pickle.dump(word2id, f)
with open('./var/IDF', 'wb') as f:
    pickle.dump(IDF, f)
    
# np.save(file = './var/TF_IDF', arr = TF_IDF_pp)

In [9]:
new_data.to_excel("./var/new_data.xlsx", sheet_name="Sheet1")

In [10]:
TF = list(TF_IDF['TF'])
with open('./var/TF', 'wb') as f:
    pickle.dump(TF, f)

In [11]:
np.savez_compressed(file = './var/TF_IDF', arr = TF_IDF_pp)

In [12]:
print(TF_IDF_pp.max())

102.15688268359305
